# Budget Analysis

---

## Environment Setup

In [28]:
# Initial Imports
import os
import plaid
import requests
import datetime
import json
import pandas as pd
from dotenv import load_dotenv

%matplotlib inline

In [29]:
# Helper functions

def pretty_print_response(response):
    """
    This function formats a JSON file to pretty printing it in the screen.
    """
    print(json.dumps(response, indent=4, sort_keys=True))

In [30]:
# Set environment variables from the .env file
load_dotenv()

True

In [31]:
# Extract API keys from environment variables
PLAID_CLIENT_ID = '5eb6c04fe3979600125b731f'
PLAID_PUBLIC_KEY = '0326e42cb20359ffe6b303bdd88a33'
PLAID_SBX_SECRET_KEY = '7a025b63458474c4b9e094ed4fc815'

---

## Plaid Access Token

In this section, you will use the `plaid-python` API to generate the correct authentication tokens to access data in the free developer Sandbox. This mimics how you might connect to your own account or a customer account, but due to privacy issues, this homework will only require connecting to and analyzing the fake data from the developer sandbox that Plaid provides. 

Complete the following steps to generate an access token:

1. Create a client to connect to plaid

2. Use the client to generate a public token and request the following items: ['transactions', 'income', 'assets']

3. Exchange the public token for an access token

4. Test the access token by requesting and printing the available test accounts

### 1. Create a client to connect to plaid

In [32]:
# Create the client object
client = plaid.Client(client_id=PLAID_CLIENT_ID, secret=PLAID_SBX_SECRET_KEY, public_key=PLAID_PUBLIC_KEY, environment='sandbox')

In [33]:
# Set the institution id
INSTITUTION_ID = "ins_109508"

### 2. Generate a public token

In [34]:
# Create the public token

create_tkn_response = client.Sandbox.public_token.create(INSTITUTION_ID, ['transactions','income','assets'])

# Print the public token

create_tkn_response


{'public_token': 'public-sandbox-5dabe725-f08b-4874-b30d-af50a9c7c209',
 'request_id': 'p8oYjSxDZrG5gDD'}

### 3. Exchange the public token for an access token

In [35]:
# Retrieve the access token

exchange_response = client.Item.public_token.exchange(create_tkn_response['public_token'])

# Print the access toke
exchange_response

{'access_token': 'access-sandbox-30110cd8-1aad-4800-a4e4-dda11ef3324d',
 'item_id': 'PVR7nbpmNNHEeMaRJ1LkIgag56ldVZC7v1gZz',
 'request_id': 'Sbxjv5QgkFkpugM'}

In [36]:
# Store the access token in a Python variable
access_token = exchange_response['access_token']

### 4. Fetch Accounts

In [37]:
# Retrieve accounts

client.Accounts.get(access_token)

# Print accounts


{'accounts': [{'account_id': 'rPXw8JVjNNI9ydjg1bALHZmZgp3jVBtyK57bl',
   'balances': {'available': 100,
    'current': 110,
    'iso_currency_code': 'USD',
    'limit': None,
    'unofficial_currency_code': None},
   'mask': '0000',
   'name': 'Plaid Checking',
   'official_name': 'Plaid Gold Standard 0% Interest Checking',
   'subtype': 'checking',
   'type': 'depository'},
  {'account_id': 'zLBzymqnooSDzxV6njmri7B7nLRJXDFEPvldl',
   'balances': {'available': 200,
    'current': 210,
    'iso_currency_code': 'USD',
    'limit': None,
    'unofficial_currency_code': None},
   'mask': '1111',
   'name': 'Plaid Saving',
   'official_name': 'Plaid Silver Standard 0.1% Interest Saving',
   'subtype': 'savings',
   'type': 'depository'},
  {'account_id': 'BXNkeRxryyhZAyMgKvreIlqlMmkP1Qt7eK4vz',
   'balances': {'available': None,
    'current': 1000,
    'iso_currency_code': 'USD',
    'limit': None,
    'unofficial_currency_code': None},
   'mask': '2222',
   'name': 'Plaid CD',
   'officia

---

# Account Transactions with Plaid

In this section, you will use the Plaid Python SDK to connect to the Developer Sandbox account and grab a list of transactions. You will need to complete the following steps:


1. Use the access token to fetch the transactions for the last 90 days

2. Print the categories for each transaction type

3. Create a new DataFrame using the following fields from the JSON transaction data: `date, name, amount, category`. (For categories with more than one label, just use the first category label in the list)

4. Convert the data types to the appropriate types (i.e. datetimeindex for the date and float for the amount)

### 1. Fetch the Transactions for the last 90 days

In [38]:
# Set the start and end date 90 days from today
start_date = '{:%Y-%m-%d}'.format(datetime.datetime.now() + datetime.timedelta(-90))
end_date = '{:%Y-%m-%d}'.format(datetime.datetime.now())

# Retrieve the transactions for the last 90 days

transaction_response = client.Transactions.get(access_token,start_date,end_date)

# Print the transactions
print(json.dumps(transaction_response['transactions'][:2],indent=4, sort_keys=True))

[
    {
        "account_id": "35VJlKgQkkUEG4XVmz6MIg7gEq95bRcwo1ZgM",
        "account_owner": null,
        "amount": 500,
        "authorized_date": null,
        "category": [
            "Food and Drink",
            "Restaurants"
        ],
        "category_id": "13005000",
        "date": "2020-05-25",
        "iso_currency_code": "USD",
        "location": {
            "address": null,
            "city": null,
            "country": null,
            "lat": null,
            "lon": null,
            "postal_code": null,
            "region": null,
            "store_number": null
        },
        "name": "Tectra Inc",
        "payment_channel": "in store",
        "payment_meta": {
            "by_order_of": null,
            "payee": null,
            "payer": null,
            "payment_method": null,
            "payment_processor": null,
            "ppd_id": null,
            "reason": null,
            "reference_number": null
        },
        "pending": false,
    

### 2. Print the categories for each transaction

In [22]:
# Create a for-loop to print the categories for each transaction
for category in transaction_response:
    print(category)

accounts
item
request_id
total_transactions
transactions


### 3. Create a new DataFrame using the following fields from the JSON transaction data: `date`, `name`, `amount`, `category`. 

(For categories with more than one label, just use the first category label in the list)

In [39]:
# Define an empty DataFrame and set the columns as date, name, amount, and category
transaction_df = pd.DataFrame(columns=['date', 'name', 'amount', 'category'])

# Retrieve all the transactions
transactions = transaction_response['transactions']

# Populate the transactions DataFrame with the transactions data
for index, transaction in enumerate(transactions):
    transaction_df.loc[index] = [transaction["date"], transaction["name"], transaction["amount"], transaction["category"][0]]


# Display sample data from the DataFrame    
transaction_df.head()

,date,name,amount,category
0,2020-05-25,Tectra Inc,500,Food and Drink
1,2020-05-24,AUTOMATIC PAYMENT - THANK,2078.5,Payment
2,2020-05-24,KFC,500,Food and Drink
3,2020-05-24,Madison Bicycle Shop,500,Shops
4,2020-05-15,CREDIT CARD 3333 PAYMENT *//,25,Payment


### 4. Convert the data types to the appropriate types 

(i.e. datetimeindex for the date and float for the amount)

In [40]:
# Display data types

transaction_df.dtypes

date        object
name        object
amount      object
category    object
dtype: object

In [41]:
# Make any required data type transformation
transaction_df ['date'] = transaction_df['date'].astype('datetime64[ns]')

transaction_df.dtypes


date        datetime64[ns]
name                object
amount              object
category            object
dtype: object

In [42]:
# Set the date column as index

transaction_df['amount'] = transaction_df['amount'].astype('float')
transaction_df['date'] = pd.to_datetime(transaction_df['date'], format='%Y-%m-%d')
transaction_df = transaction_df.set_index('date')
transaction_df.head()

# Display sample data
transaction_df.head()


,name,amount,category
date,,,
2020-05-25,Tectra Inc,500.0,Food and Drink
2020-05-24,AUTOMATIC PAYMENT - THANK,2078.5,Payment
2020-05-24,KFC,500.0,Food and Drink
2020-05-24,Madison Bicycle Shop,500.0,Shops
2020-05-15,CREDIT CARD 3333 PAYMENT *//,25.0,Payment


---

# Income Analysis with Plaid

In this section, you will use the Plaid Sandbox to complete the following:

1. Determine the previous year's gross income and print the results

2. Determine the current monthly income and print the results

3. Determine the projected yearly income and print the results

In [67]:
# Fetch the income data
income_response = client.Income.get(access_token)

# Print the income data
print(income_response)

{'income': {'income_streams': [{'confidence': 0.99, 'days': 690, 'monthly_income': 500, 'name': 'UNITED AIRLINES'}], 'last_year_income': 6000, 'last_year_income_before_tax': 7285, 'max_number_of_overlapping_income_streams': 1, 'number_of_income_streams': 1, 'projected_yearly_income': 6085, 'projected_yearly_income_before_tax': 7389}, 'request_id': 'ORsHTSKbwTgCzJA'}


In [77]:
# Determine the previous year's gross income and print the results
keys = income_response.keys()
values = income_response.values()
values

dict_values([{'income_streams': [{'confidence': 0.99, 'days': 690, 'monthly_income': 500, 'name': 'UNITED AIRLINES'}], 'last_year_income': 6000, 'last_year_income_before_tax': 7285, 'max_number_of_overlapping_income_streams': 1, 'number_of_income_streams': 1, 'projected_yearly_income': 6085, 'projected_yearly_income_before_tax': 7389}, 'ORsHTSKbwTgCzJA'])

In [ ]:
# Determine the current monthly income and print the results


In [ ]:
# Determine the projected yearly income and print the results


---

# Budget Analysis
In this section, you will use the transactions DataFrame to analyze the customer's budget

1. Calculate the total spending per category and print the results (Hint: groupby or count transactions per category)

2. Generate a bar chart with the number of transactions for each category

3. Calulate the expenses per month

4. Plot the total expenses per month

### Calculate the expenses per category

In [ ]:
# Compute the total expenses per category
transaction_df.index = pd.to_datetime((transaction_df.index)
 
transaction_by_cat = transaction_df.groupby('category').sum()

# Display the expenses per catefory
expenses_per_month = .groupby(df.index.month).sum()
                                      

In [ ]:
# Create a spending categories pie chart.


### Calculate the expenses per month

In [ ]:
# Create a DataFrame with the total expenses

# Display sample data


In [ ]:
# Create a spending per month bar chart
